In [1]:
import numpy as np
import scipy as sp
import pandas as pd

import matplotlib as mtlb
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()


In [2]:
X_train = pd.read_csv("X_train_Wwou3IE.csv")
y_train = pd.read_csv("y_train_jJtXgMX.csv")


In [14]:
X_train.head(n=5)

,DELIVERY_START,load_forecast,coal_power_available,gas_power_available,nuclear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std,predicted_spot_price
0,2022-01-01 02:00:00+01:00,49439.0,3386.0,11487.0,44118.0,3035.0,0.0,79.248348,0.0,0.0
1,2022-01-01 03:00:00+01:00,46511.0,3386.0,11487.0,44118.0,3143.0,0.0,61.776532,0.0,0.0
2,2022-01-01 04:00:00+01:00,45158.0,3386.0,11487.0,44118.0,3288.0,0.0,44.291112,0.0,0.0
3,2022-01-01 05:00:00+01:00,44779.0,3386.0,11487.0,44118.0,3447.0,0.0,36.127588,0.0,0.0
4,2022-01-01 06:00:00+01:00,45284.0,3386.0,11487.0,44118.0,3679.0,0.0,30.983023,0.0,0.0


In [4]:
X_train.describe()

,load_forecast,coal_power_available,gas_power_available,nucelear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std,predicted_spot_price
count,9318.000000,10604.000000,10604.000000,10604.000000,10581.000000,10581.000000,10581.000000,10581.000000,1846.000000
mean,53725.132968,2819.781592,11463.220671,35393.282252,4512.989415,1919.281353,125.399422,24.997384,140.256024
std,11274.194929,495.103588,483.799154,7619.820162,3349.419356,2706.963383,132.152875,43.083977,33.922669
min,30515.000000,1646.000000,9769.000000,22285.000000,592.000000,0.000000,1.830211,0.000000,42.420000
25%,45336.500000,2226.000000,11140.000000,29005.000000,2036.000000,0.000000,46.818974,0.000000,119.405000
50%,51846.000000,2806.000000,11487.000000,32470.000000,3319.000000,149.000000,86.700163,3.507229,140.040000
75%,62091.750000,3386.000000,11945.000000,43055.000000,5951.000000,3507.000000,155.325644,35.137019,160.765000
max,85145.000000,3391.000000,11963.000000,50122.000000,15167.000000,10127.000000,1871.571992,745.261316,254.190000


In [ ]:
# Rename the "nucelear_power_available" column to "nuclear_power_available" 
X_train = X_train.rename(columns={"nucelear_power_available":"nuclear_power_available"})

In [7]:
# Verify if each parameter could be a parameter of interest by counting the number of its different elements
for column in X_train.columns:
    print(f"Number of different elements in '{column}' column : {len(X_train[column].unique())}")

Number of different elements in 'DELIVERY_START' column : 10605
Number of different elements in 'load_forecast' column : 8290
Number of different elements in 'coal_power_available' column : 23
Number of different elements in 'gas_power_available' column : 66
Number of different elements in 'nucelear_power_available' column : 787
Number of different elements in 'wind_power_forecasts_average' column : 6086
Number of different elements in 'solar_power_forecasts_average' column : 4058
Number of different elements in 'wind_power_forecasts_std' column : 10582
Number of different elements in 'solar_power_forecasts_std' column : 5889
Number of different elements in 'predicted_spot_price' column : 1696


In [13]:
# The min value in the "predicted_spot_price" column is strictly upper than 0 so all the Nan values can be modifyed to 0
X_train["predicted_spot_price"] = X_train["predicted_spot_price"].fillna(0)